# Capstone East Valley Reviews Sentiment Analysis

### Imports

In [2]:
import os
import re
import io
import requests
import pandas as pd
import emojis
import string
import spacy
import nltk

In [3]:
url = "https://raw.githubusercontent.com/laurakaneta/East_Valley_Reviews/main/East_Valley_Reviews_Data_Deidentified.csv"
downloaded = requests.get(url).content
EastValleyFile = pd.read_csv(io.StringIO(downloaded.decode('ISO-8859-1')))
EastValleyFile

,ï»¿query,name,google_id,place_id,location_link,reviews_link,reviews_per_score,rating,review_id,autor_link,...,owner_answer,owner_answer_timestamp,owner_answer_timestamp_datetime_utc,review_link,review_rating,review_timestamp,review_datetime_utc,review_likes,reviews_id,City
0,"food Gilbert, AZ","Cafeâ de Bangkok- Gilbert, AZ",0x872ba94605619885:0x4d85a0e9a6d221bd,ChIJhZhhBUapK4cRvSHSpumghU0,https://www.google.com/maps/place/Cafe%E2%80%9...,https://search.google.com/local/reviews?placei...,"{""1"": 5, ""2"": 1, ""3"": 4, ""4"": 9, ""5"": 117}",4.7,ChdDSUhNMG9nS0VJQ0FnSURhbzVtaTZ3RRAB,https://www.google.com/maps/contrib/1137410538...,...,NaN,NaN,NaN,https://www.google.com/maps/reviews/data=!4m5!...,5,1630274471,08/29/2021 22:01:11,3,5586047838209581501,Gilbert
1,"food Gilbert, AZ","Cafeâ de Bangkok- Gilbert, AZ",0x872ba94605619885:0x4d85a0e9a6d221bd,ChIJhZhhBUapK4cRvSHSpumghU0,https://www.google.com/maps/place/Cafe%E2%80%9...,https://search.google.com/local/reviews?placei...,"{""1"": 5, ""2"": 1, ""3"": 4, ""4"": 9, ""5"": 117}",4.7,ChZDSUhNMG9nS0VJQ0FnSUNxeWZ1Wk13EAE,https://www.google.com/maps/contrib/1120312345...,...,NaN,NaN,NaN,https://www.google.com/maps/reviews/data=!4m5!...,3,1623383810,06/11/2021 03:56:50,0,5586047838209581501,Gilbert
2,"food Gilbert, AZ","Cafeâ de Bangkok- Gilbert, AZ",0x872ba94605619885:0x4d85a0e9a6d221bd,ChIJhZhhBUapK4cRvSHSpumghU0,https://www.google.com/maps/place/Cafe%E2%80%9...,https://search.google.com/local/reviews?placei...,"{""1"": 5, ""2"": 1, ""3"": 4, ""4"": 9, ""5"": 117}",4.7,ChdDSUhNMG9nS0VJQ0FnSURxcnFuZWh3RRAB,https://www.google.com/maps/contrib/1080138635...,...,Thank you for coming for dine in with us tonig...,1.625802e+09,07/09/2021 03:38:27,https://www.google.com/maps/reviews/data=!4m5!...,3,1625801519,07/09/2021 03:31:59,0,5586047838209581501,Gilbert
3,"food Gilbert, AZ","Cafeâ de Bangkok- Gilbert, AZ",0x872ba94605619885:0x4d85a0e9a6d221bd,ChIJhZhhBUapK4cRvSHSpumghU0,https://www.google.com/maps/place/Cafe%E2%80%9...,https://search.google.com/local/reviews?placei...,"{""1"": 5, ""2"": 1, ""3"": 4, ""4"": 9, ""5"": 117}",4.7,ChdDSUhNMG9nS0VJQ0FnSUNhMnBpMDJnRRAB,https://www.google.com/maps/contrib/1140612880...,...,NaN,NaN,NaN,https://www.google.com/maps/reviews/data=!4m5!...,5,1627531836,07/29/2021 04:10:36,1,5586047838209581501,Gilbert
4,"food Gilbert, AZ","Cafeâ de Bangkok- Gilbert, AZ",0x872ba94605619885:0x4d85a0e9a6d221bd,ChIJhZhhBUapK4cRvSHSpumghU0,https://www.google.com/maps/place/Cafe%E2%80%9...,https://search.google.com/local/reviews?placei...,"{""1"": 5, ""2"": 1, ""3"": 4, ""4"": 9, ""5"": 117}",4.7,ChZDSUhNMG9nS0VJQ0FnSUNhMW9HcUtnEAE,https://www.google.com/maps/contrib/1067087380...,...,NaN,NaN,NaN,https://www.google.com/maps/reviews/data=!4m5!...,5,1627687884,07/30/2021 23:31:24,1,5586047838209581501,Gilbert
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10457,Copper Still Moonshine Grill - Gilbert,Copper Still Moonshine Grill - Gilbert,0x872babbf7c5e9241:0xfe116985d756993b,ChIJQZJefL-rK4cRO5lW14VpEf4,https://www.google.com/maps/place/Copper+Still...,https://search.google.com/local/reviews?placei...,"{""1"": 17, ""2"": 16, ""3"": 57, ""4"": 156, ""5"": 420}",4.4,ChZDSUhNMG9nS0VJQ0FnSUR3aEx2WlhnEAE,https://www.google.com/maps/contrib/1159259620...,...,NaN,NaN,NaN,https://www.google.com/maps/reviews/data=!4m5!...,4,1539763834,10/17/2018 08:10:34,1,-139214089907431109,Gilbert
10458,Copper Still Moonshine Grill - Gilbert,Copper Still Moonshine Grill - Gilbert,0x872babbf7c5e9241:0xfe116985d756993b,ChIJQZJefL-rK4cRO5lW14VpEf4,https://www.google.com/maps/place/Copper+Still...,https://search.google.com/local/reviews?placei...,"{""1"": 17, ""2"": 16, ""3"": 57, ""4"": 156, ""5"": 420}",4.4,ChdDSUhNMG9nS0VJQ0FnSUN3OHNhNnh3RRAB,https://www.google.com/maps/contrib/1000622061...,...,NaN,NaN,NaN,https://www.google.com/maps/reviews/data=!4m5!...,5,1459199210,03/28/2016 21:06:50,1,-139214089907431109,Gilbert
10459,Copper Still Moonshine 

In [4]:
EastValleyFile.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10462 entries, 0 to 10461
Data columns (total 22 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   ï»¿query                             10462 non-null  object 
 1   name                                 10462 non-null  object 
 2   google_id                            10462 non-null  object 
 3   place_id                             10462 non-null  object 
 4   location_link                        10462 non-null  object 
 5   reviews_link                         10462 non-null  object 
 6   reviews_per_score                    10462 non-null  object 
 7   rating                               10462 non-null  float64
 8   review_id                            10462 non-null  object 
 9   autor_link                           10462 non-null  object 
 10  review_text                          10401 non-null  object 
 11  review_img_url              

Since only the reviews and ratings will be used to make the sentiment analysis, all other columns can be dropped and all na values will be dropped as well. 

In [5]:
East_Valley_slimmed_down = EastValleyFile[['review_text', 'review_rating']]
East_Valley_slimmed_down = East_Valley_slimmed_down.dropna()

In [6]:
East_Valley_slimmed_down.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10401 entries, 0 to 10461
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   review_text    10401 non-null  object
 1   review_rating  10401 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 243.8+ KB


All positive reviews (4 or 5) will be denoted with a 1, all neutral reviews (3) will be denoted with a 0, and all negative reviews (1 or 2) will be denoted with a -1 under the "sentiment" column

In [7]:
sentiment = []
for rating in East_Valley_slimmed_down.review_rating:
    if rating > 3:
        sentiment.append(1)
    elif rating < 3:
        sentiment.append(-1)
    else:
        sentiment.append(0) 
East_Valley_slimmed_down['sentiment'] = sentiment

To Pre-Process the data, all the text will be lowercase and all punctuation will be removed. (CountVectorizer actually performs these steps and tokenizes the data, but I wanted to manually inspect the reviews before using them to train the models). 

In [8]:
East_Valley_slimmed_down['review_text'] = East_Valley_slimmed_down.iloc[:,0].str.lower()

In [9]:
Reviews = East_Valley_slimmed_down['review_text'].copy()

In [10]:
Reviews_updated= Reviews.replace('['+string.punctuation+']', '', regex=True)

In [11]:
East_Valley_slimmed_down= East_Valley_slimmed_down.drop('review_text', axis = 1)
East_Valley_slimmed_down['reviews'] = Reviews_updated

I decided to remove all the reviews with a score of 3 (neutral) for the sentiment analysis. 

In [80]:
East_Valley_Negative_Positive = East_Valley_slimmed_down[East_Valley_slimmed_down.sentiment != 0]
East_Valley_Negative_Positive

,review_rating,sentiment,reviews
0,5,1,what a great find from the outside it may make...
3,5,1,the restaurant has pleasant dining seating arr...
4,5,1,been craving a thai food place since we left o...
5,5,1,amazing food with authentic flavors drunken no...
6,5,1,finally i found my favorite kuey teow nham tok...
...,...,...,...
10457,4,1,cozy atmosphere good lunch and dinner selectio...
10458,5,1,i love the neighborhood vibe of this place its...
10459,5,1,our waitress must have overdosed on vitamin d ...
10460,1,-1,poor service improperly prepared food and over...


In [13]:
East_Valley_Negative_Positive.sentiment.value_counts()

 1    8990
-1     874
Name: sentiment, dtype: int64

In [77]:
Negpercent = round(((East_Valley_Negative_Positive.sentiment.value_counts()[-1]/
                     sum(East_Valley_Negative_Positive.sentiment.value_counts()[:]))
              *100),2)

pospercent = round(((East_Valley_Negative_Positive.sentiment.value_counts()[1]/
                     sum(East_Valley_Negative_Positive.sentiment.value_counts()[:]))
              *100),2)

print(Negpercent, 'percent of the reviews have a negative sentiment, while', '\n', pospercent, 
      'percent of the reviews have a positive sentiment.')

8.86 percent of the reviews have a negative sentiment, while 
 91.14 percent of the reviews have a positive sentiment.


The data is imbalanced. Only 9.72 percent of the reviews are classified as negative sentiment. However, I do not have the ability to scrape more Google Reviews to gain more negative sentiment ones so I am going to traim multiple models and use a confusion matrix anbd a full classification report to look more in-depth at how the models performs. 

Split data to train models

In [20]:
X = East_Valley_Negative_Positive['reviews']
y = East_Valley_Negative_Positive['sentiment']

X[0]

'what a great find from the outside it may make you want to change your mind but venture inside and youll be pleasantly surprised the interior is nicely decorated and clean i believe our server was also the owner super friendly and very attentive we ordered 4 dishes the papaya salad mongolian beef green curry and vegetable curry we ordered them all medium spice level now i like some heat but the papaya salad was quite spicy next time would probably order that at a 1 otherwise everything else had a nice heat level the curries were very tasty i wasnt as big a fan on their mongolian beef dish well definitely come back here again to try some of the other dishes'

In [21]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [22]:
from sklearn.feature_extraction.text import CountVectorizer

In [81]:
cv_doc = CountVectorizer(stop_words = "english")
Xtrain_vec = cv_doc.fit_transform(X_train)
Xtest_vec = cv_doc.transform(X_test)

In [103]:
len(cv_doc.vocabulary_)

11965

In [102]:
cv_doc.vocabulary_

{'corrected': 2662,
 'order': 7417,
 'times': 10764,
 'different': 3201,
 'people': 7758,
 'got': 4751,
 'wrong': 11790,
 'told': 10820,
 'beef': 1233,
 'bowl': 1507,
 'thing': 10673,
 'italian': 5664,
 'just': 5826,
 'pile': 7894,
 'meat': 6612,
 'thats': 10644,
 'giant': 4654,
 'meatball': 6613,
 'super': 10314,
 'disappointing': 3266,
 'best': 1294,
 'coffee': 2341,
 'arizona': 849,
 'cheap': 2072,
 'drinks': 3469,
 'nice': 7138,
 'dive': 3346,
 'great': 4825,
 'place': 7934,
 'pregame': 8168,
 'wait': 11430,
 'tables': 10446,
 'party': 7665,
 'general': 4619,
 'excellent': 3882,
 'service': 9407,
 'atmosphere': 959,
 'recommend': 8582,
 'trying': 11001,
 'cheesecake': 2101,
 'store': 10152,
 'wifi': 11651,
 'access': 413,
 'downtown': 3427,
 'mesa': 6685,
 'shop': 9512,
 'junkie': 5823,
 'upstate': 11220,
 'ny': 7264,
 'gold': 4709,
 'bar': 1152,
 'little': 6235,
 'kinda': 5932,
 'reminds': 8716,
 'home': 5224,
 'espresso': 3810,
 'bean': 1219,
 'smooth': 9732,
 'medium': 6629,
 'r

## Logistic Regression

In [82]:
from sklearn.linear_model import LogisticRegression
logreg= LogisticRegression()
logreg.fit(Xtrain_vec,y_train)
preds = logreg.predict(Xtest_vec)

In [83]:
from sklearn.metrics import confusion_matrix,classification_report
confusion_matrix(preds,y_test)

array([[ 110,   13],
       [  72, 1778]])

In [84]:
print(classification_report(preds,y_test))

              precision    recall  f1-score   support

          -1       0.60      0.89      0.72       123
           1       0.99      0.96      0.98      1850

    accuracy                           0.96      1973
   macro avg       0.80      0.93      0.85      1973
weighted avg       0.97      0.96      0.96      1973



## Support Vector Machine

In [85]:
from sklearn import svm

In [86]:
clf = svm.SVC()
clf.fit(Xtrain_vec, y_train)
clfpreds = clf.predict(Xtest_vec)

In [87]:
confusion_matrix(clfpreds,y_test)

array([[  40,    0],
       [ 142, 1791]])

In [88]:
print(classification_report(clfpreds,y_test))

              precision    recall  f1-score   support

          -1       0.22      1.00      0.36        40
           1       1.00      0.93      0.96      1933

    accuracy                           0.93      1973
   macro avg       0.61      0.96      0.66      1973
weighted avg       0.98      0.93      0.95      1973



## KNeighborsClassifier

In [89]:
from sklearn.neighbors import KNeighborsClassifier

In [90]:
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(Xtrain_vec, y_train)
knnpreds = knn.predict(Xtest_vec)

In [91]:
confusion_matrix(knnpreds,y_test)

array([[   3,    0],
       [ 179, 1791]])

In [92]:
print(classification_report(knnpreds,y_test))

              precision    recall  f1-score   support

          -1       0.02      1.00      0.03         3
           1       1.00      0.91      0.95      1970

    accuracy                           0.91      1973
   macro avg       0.51      0.95      0.49      1973
weighted avg       1.00      0.91      0.95      1973



## Multinomial Naïve Bayes 

In [ ]:
from sklearn.naive_bayes import MultinomialNB

In [93]:
MNB = MultinomialNB()
MNB.fit(Xtrain_vec, y_train)
MNBpreds = MNB.predict(Xtest_vec)

In [94]:
confusion_matrix(MNBpreds,y_test)

array([[ 120,   14],
       [  62, 1777]])

In [95]:
print(classification_report(MNBpreds,y_test))

              precision    recall  f1-score   support

          -1       0.66      0.90      0.76       134
           1       0.99      0.97      0.98      1839

    accuracy                           0.96      1973
   macro avg       0.83      0.93      0.87      1973
weighted avg       0.97      0.96      0.96      1973



## Random Forest Classifier

I chose to add another model to train, the Random Forest CLassifier, since they generally perform well on imbalanced datasets. However, as we can see below from the F1 scores, it is not the best of the models trained.

In [96]:
from sklearn.ensemble import RandomForestClassifier

In [97]:
RFC = RandomForestClassifier(random_state = 42)
RFC.fit(Xtrain_vec, y_train)
y_pred_RFC = RFC.predict(Xtest_vec)

In [98]:
confusion_matrix(y_pred_RFC,y_test)

array([[  57,    3],
       [ 125, 1788]])

In [99]:
print(classification_report(y_pred_RFC,y_test))

              precision    recall  f1-score   support

          -1       0.31      0.95      0.47        60
           1       1.00      0.93      0.97      1913

    accuracy                           0.94      1973
   macro avg       0.66      0.94      0.72      1973
weighted avg       0.98      0.94      0.95      1973



### Based on the scores, particularly looking at the F1 score (-1 : 0.47 & 1: 0.97) , the Multinomial Naïve Bayes model performed the best.